In [10]:
import json
import os
import re
from pathlib import Path

import pandas as pd
import numpy as np
import omero.clients

from omero.gateway import BlitzGateway

In [47]:
password = ''

In [48]:
group = 'HTAN'
project = 'CODEX'
username = 'estorrs'
host = 'htan-imaging.wucon.wustl.edu'
port = '4064'

In [49]:
conn = BlitzGateway(username, password, host=host, port=port,
                    secure=True, group=group)
conn.connect()

True

In [50]:
project = next(iter(conn.getObjects("Project", attributes={"name": project})))

In [51]:
# list out all images in the project
dataset_to_imgs = {}
for dataset in project.listChildren():
    if dataset.getName() not in dataset_to_imgs:
        dataset_to_imgs[dataset.getName()] = {}
    for img in dataset.listChildren():
        dataset_to_imgs[dataset.getName()][img.getName()] = img
sorted(dataset_to_imgs.keys())

['2021-07-20_HT264P1',
 '2021-07-20_HT264P1_2',
 '2021-09-21_HT293N1',
 '2021-09-21_HT293N1_2',
 '2021-09-21_HT293N1_3',
 '2021-09-28_Human_Kidney_292',
 '2021-10-08_HT323B1',
 '2021-10-19_HT339B1',
 '2021-12-29_HT122P1',
 '2022-01-10_HT206B1',
 '2022-01-28_HT434P1',
 '2022-02-10_HT323B1',
 '2022-02-10_HT323B1_2',
 '2022-02-11_HT323B1',
 '2022-02-17_HT308B1',
 '2022-02-21_HT308B1',
 '2022-03-02_HT339B1',
 '2022-03-02_HT339B1_2',
 '2022-03-06_HT339B1',
 '2022-04-29_HT397B1',
 '2022-05-05_HT397B1',
 '2022-06-25_HT206B1',
 '2022-08-04_HT297B1',
 '2022-08-04_HT323B1',
 '2022-08-18_DEG_150F',
 '2022-08-18_DEG_292M',
 '2022-09-02_DEG_150F',
 '2022-09-02_DEG_292M',
 '2022-09-06_DEG_150F',
 '2022-09-06_DEG_292M',
 '2022-09-15_HT480B1',
 '2022-09-17_HT480B1',
 '2022-10-20_HT206B1__HT427P1',
 '2022-10-28_HT206B1__HT427P1',
 '2022-11-05_HT206B1',
 '2022-11-12_HT308B1__MouseMarrow_179',
 '2022-11-18_HT171B1__HT271B1',
 '2022-11-22_HT171B1',
 '2022-12-02_HT171B1__HT271B1',
 '2022-12-05_HT171B1',
 '

In [52]:
run_df = pd.read_csv('/diskmnt/Projects/Users/estorrs/sandbox/codex_tracking - codex_runs.tsv', sep='\t')
run_df

,Run Name,omero_run_name,region,Date of experiment,Name of person who performed experiment,Project,katmai qptiff filepath,storage1 qptiff filepath,katmai ome tiff filepath,Case name Specimen 1,...,Location on slide Specimen 8,Specimen 8 bbox,Case name Specimen 9,Specimen 9 name,Location on slide Specimen 9,Specimen 9 bbox,Case name Specimen 10,Specimen 10 name,Location on slide Specimen 10,Specimen 10 bbox
0,20230914_BRCA_HT206B1-H2L1-2__HT206B1-H2L1-10_...,2023-09-14_HT206B1,NaN,09/14/2023,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20230914_BRCA_HT20...,/storage1/fs1/dinglab/Active/Primary/HTAN.Imag...,/diskmnt/primary/CODEX/HTAN/20230914_BRCA_HT20...,HT206B1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20230913_BRCA_HT425B1-Extra__HT591B1-S1H2__HT5...,2023-09-13_HT425B1__HT591B1,NaN,09/13/2023,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20230913_BRCA_HT42...,/storage1/fs1/dinglab/Active/Primary/HTAN.Imag...,/diskmnt/primary/CODEX/HTAN/20230913_BRCA_HT42...,HT425B1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20230830_GBM_C3L-01046-14__C3N-01197-15__C3L-0...,2023-08-30_C3L-01046__C3L-02041__C3N-01197,NaN,08/30/2023,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20230830_GBM_C3L-0...,/storage1/fs1/dinglab/Active/Primary/HTAN.Imag...,/diskmnt/primary/CODEX/HTAN/20230830_GBM_C3L-0...,C3L-01046,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20230829_GBM_HS-16-21554_2D__HS-17-12425_3B,2023-08-29_HS-16-21554__HS-17-12425,NaN,08/29/2023,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20230829_GBM_HS-16...,/storage1/fs1/dinglab/Active/Primary/HTAN.Imag...,/diskmnt/primary/CODEX/HTAN/20230829_GBM_HS-16...,HS-16-21554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20230826_mCRC_S16-32775_Primary_and_Met,2023-08-26_S16-32775,NaN,08/26/2023,Faria,HTAN,/diskmnt/primary/CODEX/HTAN/20230826_mCRC_S16-...,/storage1/fs1/dinglab/Active/Primary/HTAN.Imag...,/diskmnt/primary/CODEX/HTAN/20230826_mCRC_S16-...,S16-32775,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,20211019-HT339B1-S1H3A1,2021-10-19_HT339B1,NaN,10/19/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20211019-HT339B1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20211019-HT339B1-S...,HT339B1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,20211008_HT323B1-S1H3A4,2021-10-08_HT323B1,NaN,10/08/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20211008_HT323B1-S...,HT323B1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,20210921_HT293N1-S1H3A1,2021-09-21_HT293N1,reg_001,09/21/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...,HT293N1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,20210921_HT293N1-S1H3A1,2021-09-21_HT293N1_2,reg_002,09/21/2021,Siqi Chen,HTAN,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...,/storage1/fs1/dinglab/Active/Projects/estorrs/...,/diskmnt/primary/CODEX/HTAN/20210921_HT293N1-S...,HT293N1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
set(run_df['omero_run_name']) - set(dataset_to_imgs.keys())

set()

In [54]:
import datetime

In [55]:
cmds = []
for i, row in run_df.iterrows():
    run_name = row['omero_run_name']
#     print(run_name)
    specimens = [row[c] for c in run_df.columns if re.findall(r'Specimen [0-9]+ name', c)]
    specimens = [x for x in specimens if not pd.isnull(x)]
    
    img_names = '___'.join(dataset_to_imgs[run_name].keys())
    
    
    for specimen in specimens:
        matches = re.findall(specimen + r'__[0-9]+.ome.tiff', img_names)
        
        if not matches:
#             print(specimen, img_names)
            date_str = run_name.split('_')[0].replace('-', '')
            fname = f'{specimen}__{date_str}.ome.tiff'
            print(fname)
    
            directory = '/'.join(row['katmai ome tiff filepath'].split('/')[:-1])
            filepath = os.path.join(directory, fname)
            cmd = f'omero import {filepath} -T Project:name:CODEX/Dataset:name:{run_name}'
            cmds.append(cmd)
    

mCRC_S16-38794_Primary__20230824.ome.tiff
 HT271B1-S1H6A5__20230420.ome.tiff
K2200591-2 __20230420.ome.tiff
p21_female__20230428.ome.tiff
Human Kidney 292__20210928.ome.tiff


In [56]:
for cmd in cmds: print(cmd)

omero import /diskmnt/primary/CODEX/HTAN/20230824_mCRC_S16-38794_Primary/mCRC_S16-38794_Primary__20230824.ome.tiff -T Project:name:CODEX/Dataset:name:2023-08-24_S16-38794
omero import /diskmnt/primary/CODEX/HTAN/041923_BRCA_HT171B1-S1H9A1-4_top_HT243B1-S1H4A4_HT271B1-S1H6A5_HT308B1-S1H5A4_btm/ HT271B1-S1H6A5__20230420.ome.tiff -T Project:name:CODEX/Dataset:name:2023-04-20_HT171B1__HT243B1__HT308B1___HT271B1
omero import /diskmnt/primary/CODEX/HTAN/042023_PKD_K2200581-3_top_K2200591-2_mid_K2200609-2_btm/K2200591-2 __20230420.ome.tiff -T Project:name:CODEX/Dataset:name:2023-04-20_K2200581-3__K2200591-2___K2200609-2
omero import /diskmnt/primary/CODEX/HTAN/042723_GUDMAP_E16.5_TOP_P1_P21_BTTM_MALE_LEFT_FEMALE_RIGHT/p21_female__20230428.ome.tiff -T Project:name:CODEX/Dataset:name:2023-04-28_E16.5_female__E16.5_male__P1_female__P1_male__P21_male__p21_female
omero import /diskmnt/primary/CODEX/HTAN/20210928-Human_Kidney_292/Human Kidney 292__20210928.ome.tiff -T Project:name:CODEX/Dataset:nam